In [19]:
import pandas as pd
import tensorflow as tf
import numpy as np
from datetime import datetime
import tempfile
from copy import deepcopy
import math

In [2]:
df_train = pd.read_csv("sample_dataset/events_user_features.csv")

In [3]:
def epoch2datetime(x):
    epoch = (x + 1465876799998) / 1000.
    tstp = datetime.fromtimestamp(epoch)
    return tstp
#     return [tstp.weekday(), tstp.hour]

In [4]:
df_train['weekday'] = df_train.timestamp.map(lambda d: epoch2datetime(d).weekday())

In [5]:
df_train['hour'] = df_train.timestamp.map(lambda d: epoch2datetime(d).hour)

In [6]:
df_train_drop = df_train.drop(['timestamp'],axis=1)

In [7]:
df_train_drop.head()

,display_id,ad_id,uuid,document_id,platform,geo_location,appeared,uuid_doc_impression,total_impression,uuid_seen_ad,clicked,weekday,hour
0,1,42337,cb8c55702adb93,379743,3,US>SC>519,2,1,6,1,0,1,11
1,1,139684,cb8c55702adb93,379743,3,US>SC>519,2,1,6,1,0,1,11
2,1,144739,cb8c55702adb93,379743,3,US>SC>519,2,1,6,1,1,1,11
3,1,156824,cb8c55702adb93,379743,3,US>SC>519,2,1,6,1,0,1,11
4,1,279295,cb8c55702adb93,379743,3,US>SC>519,2,1,6,1,0,1,11


In [8]:
CATEGORICAL_COLUMNS = ['display_id','ad_id','document_id','uuid','geo_location','platform','weekday','hour']
CONTINUOUS_COLUMNS = ['uuid_doc_impression','total_impression','uuid_seen_ad'
                      ,'appeared']
LABEL_COLUMN = 'clicked'

In [9]:
df_train.geo_location.fillna('US>CA>803',inplace=True)

In [10]:
def input_fn(df):
  # Creates a dictionary mapping from each continuous feature column name (k) to
  # the values of that column stored in a constant Tensor.
  
    continuous_cols = {k: tf.constant(df[k].values)
                     for k in CONTINUOUS_COLUMNS}
  # Creates a dictionary mapping from each categorical feature column name (k)
  # to the values of that column stored in a tf.SparseTensor.
    categorical_cols = {k: tf.SparseTensor(
      indices=[[i, 0] for i in range(df[k].size)],
      values=df[k].values,
      shape=[df[k].size, 1])
                      for k in CATEGORICAL_COLUMNS}
  # Merges the two dictionaries into one.
    feature_cols = dict(continuous_cols.items() + categorical_cols.items())
  # Converts the label column into a constant Tensor.
    label = tf.constant(df[LABEL_COLUMN].values)
  # Returns the feature columns and the label.
    return feature_cols, label

In [11]:
display_id = tf.contrib.layers.sparse_column_with_integerized_feature("display_id",bucket_size=df_train.display_id.max())
uuid = tf.contrib.layers.sparse_column_with_hash_bucket("uuid", hash_bucket_size=100000)
ad_id = tf.contrib.layers.sparse_column_with_integerized_feature("ad_id", bucket_size=df_train.ad_id.max())
document_id = tf.contrib.layers.sparse_column_with_integerized_feature("document_id",bucket_size=df_train.document_id.max())
geo_location = tf.contrib.layers.sparse_column_with_hash_bucket('geo_location',hash_bucket_size=100)
platform = tf.contrib.layers.sparse_column_with_integerized_feature('platform',bucket_size=3)
# weekday = tf.contrib.layers.real_valued_column('weekday')
weekday = tf.contrib.layers.sparse_column_with_integerized_feature('weekday',bucket_size=7)
hour = tf.contrib.layers.sparse_column_with_integerized_feature('hour',bucket_size=24)
# hour = tf.contrib.layers.real_valued_column('hour')

In [12]:
uuid_impress_per_day = tf.contrib.layers.real_valued_column('uuid_doc_impression',dtype=tf.int64,normalizer=lambda d:d/2)
uuid_doc_per_day = tf.contrib.layers.real_valued_column('total_impression')
uuid_doc_ad_per_day = tf.contrib.layers.real_valued_column('uuid_seen_ad')
day_appeared = tf.contrib.layers.real_valued_column('appeared')

In [13]:
# uuid_impress_per_day_square = tf.contrib.layers.real_valued_column('uuid_doc_impression',normalizer=lambda d: d**2)
# uuid_doc_per_day_square = tf.contrib.layers.real_valued_column('total_impression',normalizer=lambda d: d**2)
# uuid_doc_ad_square = tf.contrib.layers.real_valued_column('uuid_seen_ad',normalizer=lambda d: d**2)
# day_appeared_square = tf.contrib.layers.real_valued_column('appeared',normalizer=lambda d: d**2)

In [14]:
# uuid_impress_per_day_sqrt = tf.contrib.layers.real_valued_column('uuid_doc_impression',normalizer=lambda d: math.sqrt(d))
# uuid_doc_per_day_sqrt = tf.contrib.layers.real_valued_column('total_impression',normalizer=lambda d: math.sqrt(d))
# uuid_doc_ad_sqrt = tf.contrib.layers.real_valued_column('uuid_seen_ad',normalizer=lambda d: math.sqrt(d))
# day_appeared_sqrt = tf.contrib.layers.real_valued_column('appeared',normalizer=lambda d: math.sqrt(d))

In [15]:
# uuid_ad = tf.contrib.layers.crossed_column(
#   [uuid,ad_id], hash_bucket_size=int(1e9))
# doc_ad = tf.contrib.layers.crossed_column(
#   [document_id,ad_id], hash_bucket_size=int(1e9))
uuid_geo = tf.contrib.layers.crossed_column(combiner='sqrtn',
    columns= [uuid,geo_location], hash_bucket_size=int(1e9))

In [16]:
train = df_train_drop[:200000]
test = df_train_drop[200000:300000]

In [18]:
def train_input_fn():
    return input_fn(train[:100])
def eval_input_fn():
    return input_fn(test[100:200])

In [46]:
columns = deepcopy(CATEGORICAL_COLUMNS)
columns.extend(CONTINUOUS_COLUMNS)

In [47]:
validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    test[columns],
    test[LABEL_COLUMN],
    every_n_steps=50)

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.


# Wide model

In [45]:
import tempfile
model_dir = tempfile.mkdtemp()
linear_columns = [weekday,
hour,platform,uuid_impress_per_day,uuid_doc_per_day,uuid_doc_ad_per_day,day_appeared, display_id,ad_id,
document_id,uuid,geo_location,uuid_geo
]
m = tf.contrib.learn.LinearClassifier(feature_columns=linear_columns,
model_dir='linear_model',)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'save_summary_steps': 100, '_num_ps_replicas': 0, '_task_type': None, '_environment': 'local', '_is_chief': True, 'save_checkpoints_secs': 600, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc763d2af10>, 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, 'tf_random_seed': None, 'keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', 'save_checkpoints_steps': None, '_master': '', 'keep_checkpoint_max': 5}


In [44]:
m.fit(input_fn=train_input_fn, steps=300,monitors=[validation_monitor])

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:loss = 0.693225, step = 1
IN

ValueError: Features are incompatible with given information. Given features: Tensor("input:0", shape=(?, 3), dtype=float64), required signatures: {'display_id': TensorSignature(dtype=tf.int64, shape=None, is_sparse=True), 'ad_id': TensorSignature(dtype=tf.int64, shape=None, is_sparse=True), 'uuid': TensorSignature(dtype=tf.string, shape=None, is_sparse=True), 'hour': TensorSignature(dtype=tf.int64, shape=None, is_sparse=True), 'uuid_seen_ad': TensorSignature(dtype=tf.int64, shape=TensorShape([Dimension(200000)]), is_sparse=False), 'appeared': TensorSignature(dtype=tf.int64, shape=TensorShape([Dimension(200000)]), is_sparse=False), 'platform': TensorSignature(dtype=tf.int64, shape=None, is_sparse=True), 'geo_location': TensorSignature(dtype=tf.string, shape=None, is_sparse=True), 'weekday': TensorSignature(dtype=tf.int64, shape=None, is_sparse=True), 'total_impression': TensorSignature(dtype=tf.int64, shape=TensorShape([Dimension(200000)]), is_sparse=False), 'uuid_doc_impression': Tens

In [ ]:
results = m.evaluate(input_fn=eval_input_fn,steps=1)

# Deep model

In [122]:
deep_columns = [
  tf.contrib.layers.embedding_column(document_id, dimension=16,combiner='sqrtn'),
  tf.contrib.layers.embedding_column(ad_id, dimension=10,combiner='sqrtn'),
  tf.contrib.layers.embedding_column(uuid, dimension=16,combiner='sqrtn'),
  tf.contrib.layers.embedding_column(geo_location, dimension=4,combiner='sqrtn'),
  weekday,hour,platform,
  uuid_impress_per_day,uuid_doc_per_day,uuid_doc_ad_per_day,day_appeared]

In [123]:
import tempfile
model_dir = tempfile.mkdtemp()
m = tf.contrib.learn.DNNClassifier(feature_columns=deep_columns,hidden_units=[64],
                                   optimizer=tf.train.AdamOptimizer(),
model_dir=model_dir)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'save_summary_steps': 100, '_num_ps_replicas': 0, '_task_type': None, '_environment': 'local', '_is_chief': True, 'save_checkpoints_secs': 600, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f08c3f8c1d0>, 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, 'tf_random_seed': None, 'keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', 'save_checkpoints_steps': None, '_master': '', 'keep_checkpoint_max': 5}


In [124]:
estimator = m.fit(input_fn=train_input_fn, steps=200)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


ValueError: Error creating input layer for column: hour.
SparseColumn is not supported in DNN. Please use embedding_column or one_hot_column. column: _SparseColumn(column_name='hour', is_integerized=True, bucket_size=24, lookup_config=None, combiner='sum', dtype=tf.int64), No deep embedding lookup arguments for column _SparseColumn(column_name='hour', is_integerized=True, bucket_size=24, lookup_config=None, combiner='sum', dtype=tf.int64).

In [ ]:
results = m.evaluate(input_fn=eval_input_fn,steps=1)

# Wide and Deep model

In [264]:
import math
from copy import deepcopy

In [270]:
math.log(len(df_train.document_id.unique()))

10.11204524552192

In [127]:
wide_columns = [weekday,hour,platform,
                uuid_doc_ad_per_day,day_appeared,uuid_impress_per_day,uuid_doc_per_day,geo_location]
deep_columns = [
  tf.contrib.layers.embedding_column(document_id, dimension=10,combiner='sqrtn'),
  tf.contrib.layers.embedding_column(ad_id, dimension=10,combiner='sqrtn'),
  tf.contrib.layers.embedding_column(uuid, dimension=16,combiner='sqrtn'),
  uuid_impress_per_day,uuid_doc_per_day, uuid_doc_ad_per_day,day_appeared
]

In [131]:
model_dir = tempfile.mkdtemp()
m = tf.contrib.learn.DNNLinearCombinedClassifier(dnn_optimizer=tf.train.AdamOptimizer(),
    model_dir=model_dir,
    linear_feature_columns=wide_columns,
    dnn_feature_columns=deep_columns,
    dnn_hidden_units=[64])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'save_summary_steps': 100, '_num_ps_replicas': 0, '_task_type': None, '_environment': 'local', '_is_chief': True, 'save_checkpoints_secs': 600, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f089fe0e5d0>, 'tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_task_id': 0, 'tf_random_seed': None, 'keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', 'save_checkpoints_steps': None, '_master': '', 'keep_checkpoint_max': 5}


In [132]:
estimator = m.fit(input_fn=train_input_fn, steps=300)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scal

In [133]:
results = m.evaluate(input_fn=eval_input_fn,steps=1)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scal